# Basic EDA

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 8)
sns.set_theme(context="notebook", style="whitegrid")

## Config

In [ ]:
# file paths
DATA_DIR = Path("..", "input")

# data
TRAIN_DATA = DATA_DIR / "train.csv"

# columns in the data
INDEX_COL = "id"

TARGET_COL = "target"

# random state
RANDOM_STATE = 42

## Loading the data

In [ ]:
df = pd.read_csv(TRAIN_DATA, index_col=INDEX_COL)
df.info()

## Pandas profiling report

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df)
profile

## Separate features from target

In [ ]:
y = df[TARGET_COL]
X = df.drop(TARGET_COL, axis=1)

numeric_dtypes = ["int64", "float64"]
categorical_df = X.select_dtypes(exclude=numeric_dtypes)
numeric_df = X.select_dtypes(include=numeric_dtypes)

## Distribution of variables

### Target variable

In [ ]:
fig, ax = plt.subplots()

sns.countplot(y, ax=ax)
ax.set_title(f"Distribution of {TARGET_COL}")
plt.show()

### Numeric features

In [ ]:
numeric_cols = numeric_df.columns.tolist()
rows = np.ceil(len(numeric_cols) / 2).astype(int)
fig, axes = plt.subplots(rows, 2, figsize=(14, 8 // 2 * rows))
plt.tight_layout()

for i, col in enumerate(numeric_cols):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=df, x=col, ax=ax)
    ax.set_title(f"Histogram of {col}", y=0.88)

plt.show()

### Numeric features by target

In [ ]:
fig, axes = plt.subplots(rows, 2, figsize=(14, 8 // 2 * rows))
plt.tight_layout()

for i, col in enumerate(numeric_cols):
    ax = axes[i // 2, i % 2]
    sns.histplot(data=df, x=col, hue=TARGET_COL, element="step", ax=ax)
    ax.set_title(f"Histogram of {col} by {TARGET_COL}", y=0.88)

plt.show()

### Categorical features

In [ ]:
for col in categorical_df.columns.tolist():
    print(f"Distribution of {col}")
    print(X[col].value_counts())
    print()

In [ ]:
categorical_cols = categorical_df.columns.tolist()
rows = np.ceil(len(categorical_cols) / 2).astype(int)
fig, axes = plt.subplots(rows, 2, figsize=(14, 8 // 2 * rows))
plt.tight_layout()

for i, col in enumerate(categorical_cols):
    ax = axes[i // 2, i % 2]
    sns.countplot(data=X, x=col, ax=ax)
    ax.set_title(f"Count of {col} classes", y=0.88)

plt.show()

## Handling missing values

In [ ]:
df.isna().sum()

In [ ]:
categorical_df = categorical_df.fillna(np.squeeze(categorical_df.mode()))
numeric_df = numeric_df.fillna(numeric_df.mean())
X = categorical_df.join(numeric_df)
any(X.isna().sum() > 0)

## Encoding categorical features

In [ ]:
for col in categorical_df.columns:
    X[col], _ = X[col].factorize()

X[categorical_df.columns].info()

## Mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_classif

discrete_features = X.columns.isin(categorical_df.columns)
mi_scores = mutual_info_classif(
    X, y, discrete_features=discrete_features, random_state=RANDOM_STATE
)

mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(14, len(X.columns)//3))

sns.barplot(x=mi_scores.values, y=mi_scores.index, ax=ax, color="tab:blue")
ax.set_title("Mutual information scores")
plt.show()